# Packages 

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from branca.colormap import linear
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import statsmodels.formula.api as smf
from sklearn import metrics
from sklearn.model_selection import train_test_split
from statsmodels.stats.diagnostic import het_white , normal_ad
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, max_error, mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score, ShuffleSplit, KFold

# Import data

In [2]:
HOME_DIR = Path.cwd()
DATA_DIR = Path(HOME_DIR)
Yield_TI_data = pd.read_csv("E:\SAMS\datasets\R\spam2020V1r0_global_Y_TR.csv", sep=",")

In [16]:
print(Yield_TI_data.shape)
print(Yield_TI_data.head())

(501831, 506)
            x          y  bana_r  barl_r  bean_r  cass_r  chic_r  citr_r  \
0 -117.125000  69.958298     0.0     0.0     0.0     0.0     0.0     0.0   
1   59.625000  69.958298     0.0     0.0     0.0     0.0     0.0     0.0   
2   60.208301  69.958298     0.0     0.0     0.0     0.0     0.0     0.0   
3   60.291699  69.958298     0.0     0.0     0.0     0.0     0.0     0.0   
4   59.958301  69.875000     0.0     0.0     0.0     0.0     0.0     0.0   

   cnut_r  coco_r  ...     WS2M2     WS2M3     WS2M4     WS2M5     WS2M6  \
0     0.0     0.0  ...  4.601562  4.671875  3.734375  3.570312  2.578125   
1     0.0     0.0  ...  5.937500  5.421875  4.632812  5.554688  4.695312   
2     0.0     0.0  ...  5.906250  5.359375  4.664062  5.265625  4.398438   
3     0.0     0.0  ...  5.906250  5.359375  4.664062  5.265625  4.398438   
4     0.0     0.0  ...  5.906250  5.359375  4.664062  5.265625  4.398438   

      WS2M7     WS2M8     WS2M9    WS2M10    WS2M11  
0  2.859375  4.070

In [17]:
Yield_TI_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501831 entries, 0 to 501830
Columns: 506 entries, x to WS2M11
dtypes: float64(494), object(12)
memory usage: 1.9+ GB


# Exploration

In [18]:
#Drop line with NA
Yield_TI_data_0 = Yield_TI_data.dropna(how='any')
Yield_TI_data_0.shape 

(497611, 506)

In [19]:
# Informations lost
Ncol0 = Yield_TI_data.shape[0]
Ncol1= Yield_TI_data_0.shape[0]
print((Ncol0-Ncol1)*100/Ncol0)

0.840920548949746


In [22]:
Vars_drop = [
    'bana_r', 'barl_r', 'bean_r', 'cass_r', 'chic_r', 'citr_r', 'cnut_r', 'coco_r', 'coff_r', 'cott_r',
    'cowp_r', 'grou_r', 'lent_r', 'ocer_r', 'ofib_r', 'oilp_r', 'onio_r', 'ooil_r',
    'opul_r', 'orts_r', 'pige_r', 'plnt_r', 'rape_r', 'rcof_r', 'rest_r', 'rubb_r',
    'sesa_r', 'soyb_r', 'sugb_r', 'sugc_r', 'sunf_r', 'teas_r', 'temf_r', 'toba_r',
    'toma_r', 'trof_r', 'vege_r', 'yams_r', 'lon', 'lat'
]
Yield_TI_data_1 =Yield_TI_data_0.drop(columns=[var for var in Vars_drop if var in Yield_TI_data_0.columns])
Yield_TI_data_1.rename(columns={'x': 'lon', 'y': 'lat'}, inplace=True)

In [23]:
Yield_TI_data_1

,lon,lat,maiz_r,mill_r,pmil_r,pota_r,rice_r,sorg_r,swpo_r,whea_r,...,WS2M2,WS2M3,WS2M4,WS2M5,WS2M6,WS2M7,WS2M8,WS2M9,WS2M10,WS2M11
649,-139.292007,67.958298,0.0,0.0,0.0,52870.4,0.0,0.0,0.0,0.0,...,3.328125,2.718750,2.804688,2.648438,2.445312,2.148438,2.601562,2.281250,2.656250,2.359375
650,-139.207993,67.958298,0.0,0.0,0.0,7603.0,0.0,0.0,0.0,0.0,...,3.328125,2.718750,2.804688,2.648438,2.445312,2.148438,2.601562,2.281250,2.656250,2.359375
651,-139.125000,67.958298,0.0,0.0,0.0,7603.0,0.0,0.0,0.0,0.0,...,3.328125,2.718750,2.804688,2.648438,2.445312,2.148438,2.601562,2.281250,2.656250,2.359375
652,-139.042007,67.958298,0.0,0.0,0.0,7603.0,0.0,0.0,0.0,0.0,...,3.460938,2.789062,2.703125,2.562500,2.335938,2.054688,2.507812,2.328125,2.632812,2.562500
653,-138.957993,67.958298,0.0,0.0,0.0,52870.4,0.0,0.0,0.0,0.0,...,3.460938,2.789062,2.703125,2.562500,2.335938,2.054688,2.507812,2.328125,2.632812,2.562500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501826,169.207993,-46.541699,3946.7,0.0,0.0,92375.1,0.0,0.0,10218.7,2345.9,...,3.781250,4.617188,3.179688,4.093750,4.210938,4.257812,4.773438,4.062500,3.859375,4.593750
501827,169.042007,-46.625000,3947.5,0.0,0.0,114909.5,0.0,0.0,12711.5,2854.1,...,3.460938,4.148438,2.906250,3.578125,3.601562,3.742188,4.562500,3.914062,3.617188,4.343750
501828,169.125000,-46.625000,3946.4,0.0,0.0,124177.1,0.0,0.0,13736.7,2708.3,...,3.781250,4.617188,3.179688,4.093750,4.210938,4.257812,4.773438,4.062500,3.859375,4.593750
501829,169.207993,-46.625000,3917.5,0.0,0.0,117353.3,0.0,0.0,12981.8,2689.8,...,3.781250,4.617188,3.179688,4.093750,4.210938,4.257812,4.773438,4.062500,3.859375,4.593750


In [24]:
SKY_groups = {
    "SKY_1": [col for col in Yield_TI_data_1.columns if col.startswith("ALLSKY_KT")],
    "SKY_2": [col for col in Yield_TI_data_1.columns if col.startswith("ALLSKY_SRF_ALB")],
    "SKY_3": [col for col in Yield_TI_data_1.columns if col.startswith("ALLSKY_SFC_LW_DWN")],
    "SKY_4": [col for col in Yield_TI_data_1.columns if col.startswith("ALLSKY_SFC_LW_UP")],
    "SKY_5": [col for col in Yield_TI_data_1.columns if col.startswith("ALLSKY_SFC_PAR_TOT")],
    "SKY_6": [col for col in Yield_TI_data_1.columns if col.startswith("ALLSKY_SFC_SW_DWN")],
    "SKY_7": [col for col in Yield_TI_data_1.columns if col.startswith("ALLSKY_SFC_SW_UP")],
    "SKY_8": [col for col in Yield_TI_data_1.columns if col.startswith("ALLSKY_SFC_UV_INDEX")],
    "SKY_9": [col for col in Yield_TI_data_1.columns if col.startswith("ALLSKY_SFC_UVA")],
    "SKY_10": [col for col in Yield_TI_data_1.columns if col.startswith("ALLSKY_SFC_UVB")],
    "SKY_12": [col for col in Yield_TI_data_1.columns if col.startswith("CLRSKY_DAYS")],
    "SKY_13": [col for col in Yield_TI_data_1.columns if col.startswith("CLRSKY_SRF_ALB")],
    "SKY_14": [col for col in Yield_TI_data_1.columns if col.startswith("CLRSKY_SFC_LW_DWN")],
    "SKY_15": [col for col in Yield_TI_data_1.columns if col.startswith("CLRSKY_SFC_LW_UP")],
    "SKY_16": [col for col in Yield_TI_data_1.columns if col.startswith("CLRSKY_SFC_PAR_TOT")],
    "SKY_17": [col for col in Yield_TI_data_1.columns if col.startswith("CLRSKY_SFC_SW_DWN")],
    "SKY_18": [col for col in Yield_TI_data_1.columns if col.startswith("CLRSKY_SFC_SW_UP")],
    "CLOUD_1": [col for col in Yield_TI_data_1.columns if col.startswith("CLOUD_AMT")],
    "CLOUD_2": [col for col in Yield_TI_data_1.columns if col.startswith("CLOUD_AMT_DAY")],
    "CLOUD_3": [col for col in Yield_TI_data_1.columns if col.startswith("CLOUD_AMT_NIGHT")],
    "PW": [col for col in Yield_TI_data_1.columns if col.startswith("PW")],
    "TS": [col for col in Yield_TI_data_1.columns if col.startswith("TS")],
    "U2M": [col for col in Yield_TI_data_1.columns if col.startswith("U2M")],
    "EVLAND": [col for col in Yield_TI_data_1.columns if col.startswith("EVLAND")],
    "EVPTRNS": [col for col in Yield_TI_data_1.columns if col.startswith("EVPTRNS")],
    "V2M": [col for col in Yield_TI_data_1.columns if col.startswith("V2M")],
    "GWETPROF": [col for col in Yield_TI_data_1.columns if col.startswith("GWETPROF")],
    "RH2M": [col for col in Yield_TI_data_1.columns if col.startswith("RH2M")],
    "GWETROOT": [col for col in Yield_TI_data_1.columns if col.startswith("GWETROOT")],
    "QV2M": [col for col in Yield_TI_data_1.columns if col.startswith("QV2M")],
    "PS": [col for col in Yield_TI_data_1.columns if col.startswith("PS")],
    "Z0M": [col for col in Yield_TI_data_1.columns if col.startswith("Z0M")],
    "GWETTOP": [col for col in Yield_TI_data_1.columns if col.startswith("GWETTOP")],
    "T2M": [col for col in Yield_TI_data_1.columns if col.startswith("T2M")],
    "WD2M": [col for col in Yield_TI_data_1.columns if col.startswith("WD2M")],
    "WS2M": [col for col in Yield_TI_data_1.columns if col.startswith("WS2M")]
}

In [25]:
def process_prectotcorr(X):
    X = X.copy()  # Assurez-vous de ne pas modifier l'original
    var_cols = [col for col in X.columns if col.startswith("PRECTOTCORR")]
    X["mean_PRECTOTCORR"] = X[var_cols].fillna(0).sum(axis=1)
    X = X.loc[:, ~X.columns.str.startswith("PRECTOTCORR")]
    return X

prectotcorr_transformer = FunctionTransformer(process_prectotcorr)

def extract_numeric_values(X):
    columns_clrsky = [col for col in X.columns if col.startswith("CLRSKY_DAYS")]
    for col in columns_clrsky:
        X[col] = X[col].str.extract(r'(\d+)').astype(float)
    return X
numeric_extraction_transformer = FunctionTransformer(extract_numeric_values)

In [26]:
transformers = []
for name, columns in SKY_groups.items():
    transformer = Pipeline([
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=0.80, svd_solver='full'))  # Garde 80% of the variance
    ])
    transformers.append((name, transformer, columns))


In [27]:
preprocessor = Pipeline([
    ("prectotcorr_transformer",FunctionTransformer(process_prectotcorr)),
    ("numeric_extraction", numeric_extraction_transformer),  
    ("feature_transform", ColumnTransformer(transformers))   
])

In [72]:
Vars = ["rice_r", "whea_r","maiz_r","pmil_r","mill_r","sorg_r","pota_r","swpo_r"]

def CropSelect(crop, var, data):
    columns_to_drop = np.array(var)
    if crop in columns_to_drop:
        columns_to_drop = np.delete(columns_to_drop, np.where(columns_to_drop == crop))
    data = data.drop(columns_to_drop, axis=1)
    return data

crop ="swpo_r"
data_model = CropSelect(crop, Vars,Yield_TI_data_1)
data_model = data_model.loc[data_model[crop].notnull()]
data_model = data_model[data_model[crop]>0]
X = data_model.drop([crop], axis=1)
y = data_model.loc[:, crop]
X = X.drop("Unnamed: 0", axis=1, errors="ignore")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [41]:
data_model

,lon,lat,whea_r,ALLSKY_KT0,ALLSKY_KT1,ALLSKY_KT2,ALLSKY_KT3,ALLSKY_KT4,ALLSKY_KT5,ALLSKY_KT6,...,WS2M2,WS2M3,WS2M4,WS2M5,WS2M6,WS2M7,WS2M8,WS2M9,WS2M10,WS2M11
1577,13.208300,65.958298,4032.1,0.257812,0.320312,0.289062,0.437500,0.453125,0.523438,0.406250,...,4.734375,3.328125,2.726562,1.765625,2.023438,2.046875,3.343750,2.843750,4.156250,3.867188
1672,13.291700,65.875000,4032.1,0.257812,0.320312,0.289062,0.437500,0.453125,0.523438,0.406250,...,4.734375,3.328125,2.726562,1.765625,2.023438,2.046875,3.343750,2.843750,4.156250,3.867188
1673,24.291700,65.875000,1825.9,0.312500,0.406250,0.500000,0.500000,0.546875,0.585938,0.437500,...,1.250000,0.945312,0.585938,0.617188,0.640625,0.703125,0.843750,0.773438,1.132812,1.085938
1769,13.291700,65.791702,4032.1,0.257812,0.320312,0.289062,0.437500,0.453125,0.523438,0.406250,...,4.734375,3.328125,2.726562,1.765625,2.023438,2.046875,3.343750,2.843750,4.156250,3.867188
2684,12.125000,65.291702,4032.1,0.257812,0.320312,0.289062,0.437500,0.453125,0.523438,0.406250,...,6.218750,4.320312,3.578125,2.210938,2.398438,2.609375,4.601562,3.976562,5.851562,5.601562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501826,169.207993,-46.541699,2345.9,0.507812,0.500000,0.476562,0.460938,0.492188,0.468750,0.429688,...,3.781250,4.617188,3.179688,4.093750,4.210938,4.257812,4.773438,4.062500,3.859375,4.593750
501827,169.042007,-46.625000,2854.1,0.507812,0.500000,0.476562,0.460938,0.492188,0.468750,0.429688,...,3.460938,4.148438,2.906250,3.578125,3.601562,3.742188,4.562500,3.914062,3.617188,4.343750
501828,169.125000,-46.625000,2708.3,0.507812,0.500000,0.476562,0.460938,0.492188,0.468750,0.429688,...,3.781250,4.617188,3.179688,4.093750,4.210938,4.257812,4.773438,4.062500,3.859375,4.593750
501829,169.207993,-46.625000,2689.8,0.507812,0.500000,0.476562,0.460938,0.492188,0.468750,0.429688,...,3.781250,4.617188,3.179688,4.093750,4.210938,4.257812,4.773438,4.062500,3.859375,4.593750


# Modeling

## Decision tree model

In [30]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
model_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("decision tree model", DecisionTreeRegressor(max_depth=9)) 
])

model_pipeline.fit(X_train, y_train)

In [ ]:
y_pred_train = model_pipeline.predict(X_train)
y_pred_test = model_pipeline.predict(X_test)

In [ ]:
print(f"score train :  {r2_score(y_train, y_pred_train)}")
print(f"score test : {r2_score(y_test, y_pred_test)}")

score train :  0.6755633823044009
score test : 0.6891706748572234


## XG BOOST

In [45]:
#pip install xgboost


     -------------------------------------- 124.9/124.9 MB 4.4 MB/s eta 0:00:00


In [34]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [73]:
xgb_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("xgb",  xgb.XGBRegressor(n_estimators=6, random_state=42, max_depth=16)) 
])

xgb_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('prectotcorr_transformer',
                                  FunctionTransformer(func=<function process_prectotcorr at 0x0000023A04CA1F80>)),
                                 ('numeric_extraction',
                                  FunctionTransformer(func=<function extract_numeric_values at 0x0000023A042B1A80>)),
                                 ('feature_transform',
                                  ColumnTransformer(transformers=[('SKY_1',
                                                                   Pipeline(steps=[('scaler',
                                                                                    Standard...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=16, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=6, n_jobs=None,
                              num_parallel_tree=None, random_state=42, ...))])

In [74]:
y_pred_train = xgb_pipeline.predict(X_train)
y_pred_test = xgb_pipeline.predict(X_test)

In [75]:
print(f"score train :  {r2_score(y_train, y_pred_train)}")
print(f"score test : {r2_score(y_test, y_pred_test)}")

score train :  0.8988867168947975
score test : 0.8651994323769188


In [ ]:
scores = cross_val_score(xgb_pipeline, X_train, y_train, cv=5)

In [ ]:
print(f"Scores de la cross-validation : {scores}")
print(f"Score moyen de la cross-validation : {scores.mean()}")

Scores de la cross-validation : [0.94198916 0.94595526 0.94339684 0.95380085 0.94562766]
Score moyen de la cross-validation : 0.9461539516578051


In [76]:
import joblib

# Enregistrez le pipeline
joblib.dump(xgb_pipeline, 'swpo_R.pkl')

['swpo_R.pkl']